# <center> <h1> Testing </h1> </center>

<img src="../images/tests.png" width="200">

_TODO :_
- mettre la référence de la page wikipédia
- mettre la fonction plot proba dans un script
- écrire une introduction

In [ ]:
get_ipython().magic(u'matplotlib inline')
%run -i ../utils/credentials.py
%run -i ../utils/imports.py
%run -i ../utils/plots.py
%run -i ../utils/stats.py

# Le principe

On cherche à valider ou invalider une hypothèse $H_0$ face à une hypothèse alternative. Pour cela on on procède en plusieurs étapes : 
* On pose l'hypothèse nulle $H_0$ et l'hypothèse alternative $H_1$
* Ensuite on calcule une statistique de test notée $S$, qui est une variable aléatoire dont on connaît la loi de probabilité si H_0 est vraie
* On calcule la réalisation de cette variable aléatoire que l'on note $S_{obs}$
* On calcule la probabilité d'obtenir cette valeur si $H_0$ est vérifiée pour décider si cette hypothèse est crédible ou non

In [ ]:
loi = scs.norm(0,1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

def plot_proba(S, 
               label_pos, 
               limite_name, 
               stat_name="S", 
               sym = False,
               message = True,
               message_1 = False,
               message_2 = False,
               loi = scs.norm(0,1)):
    
    sigma = loi.std()
    mean = loi.mean()
    a = S
    b = a + sigma*8
    
    if not message :
        message_1 = ""
        message_2 = ""
    else:
        if not message_1:
            message_1 = r"$P({} < ${})".format(stat_name,limite_name)
        if not message_2:
            message_2 = r"$P({} < -${})".format(stat_name,limite_name)

    
    # integral limits
    x = np.linspace(mean-4*sigma,mean+4*sigma)
    y = loi.pdf(x)

    fig, ax = plt.subplots()
    ax.plot(x, y, 'r', linewidth=2)
    ax.set_ylim(bottom=0)

    # Make the shaded region
    ix = np.linspace(a, b)
    iy = loi.pdf(ix)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.8', edgecolor='0.5')
    ax.add_patch(poly)
    
    ax.text(label_pos, 0.15 * (np.max(y)), message_1,
            horizontalalignment='center', fontsize=10)


    if sym :
        ix_sym = np.linspace(mean-b, mean-a)
        iy_sym = loi.pdf(ix_sym)
        verts_sym = [(mean-b, 0), *zip(ix_sym, iy_sym), (mean-a, 0)]
        poly_sym = Polygon(verts_sym, facecolor='0.8', edgecolor='0.5')
        ax.add_patch(poly_sym)
        
        ax.text(mean-label_pos, 0.15 * (np.max(y)), message_2 ,
            horizontalalignment='center', fontsize=10)

    fig.text(0.9, 0.05, 'Values')
    fig.text(0.1, 0.9, 'Density')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticks([])
    ax.set_xticklabels(('$a$', '$b$'))
    ax.set_yticks([])

In [ ]:
plot_proba(S = 1.96,
           label_pos = 2.8,
           limite_name = "$S_{obs}$",
           stat_name = "S",
           sym = False)

Pour déterminer si on accepte ou non $H_0$ on détermine un seuil (souvent 5% ou 1%) en deça duquel on rejette $H_0$.

_Par exemple :_

* Si $|S_{Obs}| > 1,96 \text{  alors  } P(S>S_{Obs})<0.05 \text{ et on rejette } H_0$
* Si $|S_{Obs}| < 1,96 \text{  alors  } P(S>S_{Obs})>0.05 \text{ et on accepte } H_0$

In [ ]:
plot_proba(S = 1.96,
           label_pos = 2.8,
           limite_name = "$|S_{obs}|$",
           stat_name = "S",
           sym = True,
           message_1 = "zone de rejet",
           message_2= "zone de rejet")

Le test peut être unilatéral ou bilatéral.

__Exemple__

_Test unilatéral_ 

On veut tester si la valeur d'une moyenne est inférieur ou égale à un nombre $\mu$. On ne va rejeter l'hypothèse nulle que lorsque la moyenne empirique $\bar{x}$ est significativement plus grand que $\mu$. On a alors :

* $H_0 : E[X]=\mu$
* $H_1 : E[X]>\mu$

In [ ]:
plot_proba(S = 1.64,
           label_pos = 2.8,
           limite_name = "$|S_{obs}|$",
           stat_name = "S",
           sym = False,
           message_1 = "zone de rejet",
           message_2= "zone de rejet")

_Test bilatéral_ 

On veut tester si la valeur d'une moyenne est différente de $\mu$. On va rejeter l'hypothèse nulle dès que la moyenne empirique $\bar{x}$ est significativement différente de $\mu$. On a alors :

* $H_0 : E[X]=\mu$
* $H_1 : E[X] \neq \mu$

In [ ]:
plot_proba(S = 1.96,
           label_pos = 2.8,
           limite_name = "$|S_{obs}|$",
           stat_name = "S",
           sym = True,
           message_1 = "zone de rejet",
           message_2= "zone de rejet")

## Mise en pratique d'un test d'indépendance du $\chi^2$

Lorsque l'on a des variables qualitatives on peut vouloir tester leur indépendance deux à deux pour savoir si des caractéristiques son liées entre elles.

Si on a deux variables X et Y, dont les modalités sont respectivement $(X_1,X_2,X_3)$ et $(Y_1,Y_2)$, on veut donc savoir si le groupe des individus ayant la caractéristique $Y_1$ et celui des individus ayant la caractéristique $Y_2$ se répartissent de la même manière selon la variable $X$.

__NB__ : le problème est symétrique, on peut donc chercher à savoir si les sous-groupes selon la variables X se répartissent de la même manière selon la variable Y.

On note $I$ l'ensemble des modalités de $X$ et $J$ l'ensemble des modalités de $Y$.

#### Use case

Ici on souhaite savoir si la répartition du nombre de kill (variable quantitiative considérée ici comme une variable quatégorielle) est statistiquement la même pour les parties de type 'solo-fpp' ou les parties de type 'squade'.

In [ ]:
data_path = "../data/PUBG_train_sample.csv"
data_PUBG = pd.read_csv(data_path)
data_PUBG = data_PUBG.loc[data_PUBG["matchType"].isin(["solo-fpp","squad"]),:]

In [ ]:
cont = pd.crosstab(data_PUBG.matchType, data_PUBG.killStreaks)
cont

### 1/ Table de contingence en cas d'indépendance parfaite

Pour chaque couple possible $(i,j)$ on note $O_{ij}$ le nombre d'individus pour lesquel la caractéristique $X$ vaut $X_i$ et la caractéristique $Y$ vaut $Y_j$.

Pour chaque couple $(i,j)$, on commence par calculer la répartition théorique en cas d'indépendance parfaite (notée $E_{ij}$). Pour chaque couple de modalité $(i,j)$ (par exemple (killStreaks = 3, matchType= "squad"), cette répartition est donnée par :

$$E_{ij} = \frac{O_{i+} \times O_{+j}}{N}$$

Avec :
- $O_{i+} = \sum_{j}O_{ij}$ : le nombre d'individus pour lesquel la caractéristique $X$ vaut $X_i$
- $O_{+j} = \sum_{i}O_{ij}$ : le nombre d'individus pour lesquel la caractéristique $Y$ vaut $Y_j$
- $N$ : le nomre d'individu total

In [ ]:
N = cont.sum(axis=1).sum(axis=0)
E_table = pd.DataFrame(columns=cont.columns, index=cont.index)

for i in range(len(E_table)):
    for j in range(len(E_table.columns)):
        E_table.iloc[i,j] = (cont.iloc[:,j].sum()*cont.iloc[i,:].sum())/N

In [ ]:
E_table

### 2/ Distance entre la table théorique et la table réelle

Ensuite on compare cette répartition théorique idéale en cas d'indépendance avec la répartition réelle. On note $T$ la valeur de la distance entre les deux situations :
$$T = \sum_{ij}\frac{(O_{ij}-E_{ij})^2}{E_{ij}} $$

In [ ]:
T_ij = ((E_table - cont)**2)/E_table
T = dist_ij.sum(axis=0).sum()

In [ ]:
T

### 3/ Utilisier la théorie statistique pour déterminer la loi de T sous l'hypothèse d'indépendance

On fait deux hypothèses :
- $H_0$ : X et Y sont des variables indépendantes
- $H_1$ : X et Y sont liées

Or on sait que sous l'hypothèse $H_0$ on a :
$$T \sim \chi^2 (M)$$

Avec : 
$M = (card(I)-1)(card(J)-1)$

In [ ]:
loi_T = scs.chi2(8)
confidence = 0.95

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
plot_proba(S = loi_T.ppf(confidence),
           label_pos = 2.8,
           sym = False,
           limite_name = "",
           message = False,
           stat_name = "",
           loi = loi_T)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plot_proba(S = loi_T.ppf(confidence),
           label_pos = 2.8,
           sym = False,
           limite_name = "",
           message = False,
           stat_name = "",
           loi = loi_T)

plt.plot((T,T),(0,1))
plt.show()

### 4/ décider si on retient l'hypothèse nulle ou non

In [ ]:
import scipy.stats as st
st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(contingency_table)

In [ ]:
round(st_p*100,200)

<center> <h1>Take away</h1> </center>

<img src="./images/coffee.png" width="200">


__Expresso__ : 

* point 1
* point 2

__Sugar Story__ :

* point 1
* point 2

# Get more on my github <img src="./images/github.png" width="100">
https://github.com/JJublanc/statistics_tools